In [1]:
!export PYTHONPATH=${PYTHONPATH}:/home/sai/PythonRobotics/PathPlanning/CubicSpline

In [49]:
from ppo_control import PPOControl
import cubic_spline_planner
import matplotlib.pyplot as plt
import time
import numpy as np
import math
%matplotlib tk

In [50]:
husky_ppo = PPOControl()

In [5]:
ax = [0.0,6.0, 12.5, 14.0, 17.5, 20.0, 25.0] 
ay = [0.0,-3.0, -5.0, -4, 3.0, 2, 0.0]
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(ax, ay, ds= 0.5)
plt.plot(cx, cy, "+")
plt.show()
cv = [1.0]*len(cx)
husky_ppo.set_waypoints_from_list(cx, cy, cyaw, cv)

In [51]:
#husky_ppo.read_waypoint_file("./unity_waypoints_bkp.txt")
#husky_ppo.read_waypoint_file("ww.txt")
#husky_ppo.read_waypoint_file("./unity_flat.txt")
husky_ppo.read_waypoint_file("./planner_way.txt")
cx = [i[0] for i in husky_ppo.waypoints_list]
cy = [i[1] for i in husky_ppo.waypoints_list]
plt.plot(cx, cy, "+")
plt.show()

In [52]:
def simulate_warthog(X, v, w, dt):
    xcurr = X[0] + v*math.cos(X[2])*dt
    ycurr = X[1] + v*math.sin(X[2])*dt
    thcurr = X[2]+w*dt
    return [xcurr, ycurr, thcurr]

In [18]:
def simulate_warthog2(X, v, w, dt, v_prev, w_prev):
    xcurr = X[0] + v_prev*math.cos(X[2])*dt
    ycurr = X[1] + v_prev*math.sin(X[2])*dt
    thcurr = X[2]+w_prev*dt
    alpha = 1
    beta = 1
    vcurr = v*(alpha) + v_prev*(1-alpha)
    wcurr = w*(alpha) + w_prev*(1-alpha)
    vcurr = v*(alpha)
    wcurr = w*(beta)
    return [xcurr, ycurr, thcurr, vcurr, wcurr]

In [53]:
#husky_ppo.set_pose([30.67, -33.33, 3.06])
#husky_ppo.set_pose([120.180, -78.957, 140*math.pi/180.])
#husky_ppo.set_pose([132.180, -78.957, 160*math.pi/180.])
husky_ppo.set_twist([0. , 0.])
#husky_ppo.set_pose([0. , 0.5, 7*math.pi/4])
start_idx = 0
husky_ppo.set_pose([cx[start_idx]+0.05 , cy[start_idx] + 0.05, husky_ppo.waypoints_list[start_idx][2]])
husky_ppo.read_ppo_policy("./model1.zip")
#husky_ppo.read_tf_frozen_graph("/home/sai/hdd1/ml-master/ml-agents/config/ppo/results/wlong_path52/3DBall/frozen_graph_def.pb")

Using cpu device
Wrapping the env in a DummyVecEnv.


In [54]:
x_pose = []
y_pose = []
v_prev = 0
w_prev = 0
for i in range(0, 2000):
    obs = husky_ppo.get_observation()
    twist = husky_ppo.get_ppo_control(np.array(obs).reshape(1,42))[0]
    print(twist)
    v = np.clip(twist[0][0], 0, 1) * 4.0
    w = np.clip(twist[0][1], -1, 1) * 2.5
    #current_pose = simulate_warthog2(husky_ppo.get_pose(), v, w, 0.05, v_prev, w_prev)
    current_pose = simulate_warthog(husky_ppo.get_pose(), v, w, 0.05)
    print(twist[0])
    closest_idx = husky_ppo.closest_idx
    print(closest_idx, "\n")
    #husky_ppo.set_pose(current_pose[0:3])
    husky_ppo.set_pose(current_pose)
    #v_prev = current_pose[3]
    #w_prev = current_pose[4]
    husky_ppo.set_twist([v, w])
    x_pose.append(current_pose[0])
    y_pose.append(current_pose[1])
    #print(husky_ppo.closest_idx, husky_ppo.closest_dist)

[[1.        0.5079675]]
[1.        0.5079675]
0 

[[ 1. -1.]]
[ 1. -1.]
0 

[[ 0.85347635 -1.        ]]
[ 0.85347635 -1.        ]
1 

[[ 1. -1.]]
[ 1. -1.]
1 

[[ 1. -1.]]
[ 1. -1.]
1 

[[ 1. -1.]]
[ 1. -1.]
1 

[[ 0.5316817 -1.       ]]
[ 0.5316817 -1.       ]
2 

[[1.         0.45797575]]
[1.         0.45797575]
2 

[[-0.22421706  1.        ]]
[-0.22421706  1.        ]
2 

[[1.         0.89060545]]
[1.         0.89060545]
2 

[[-1.         -0.03541162]]
[-1.         -0.03541162]
3 

[[1.         0.76617837]]
[1.         0.76617837]
3 

[[-1.       -0.331863]]
[-1.       -0.331863]
3 

[[1.         0.67958343]]
[1.         0.67958343]
3 

[[-1.         -0.33186173]]
[-1.         -0.33186173]
3 

[[1.        0.6664312]]
[1.        0.6664312]
3 

[[-1.         -0.34720817]]
[-1.         -0.34720817]
3 

[[1.        0.6654579]]
[1.        0.6654579]
3 

[[-1.         -0.83426285]]
[-1.         -0.83426285]
4 

[[1.         0.12157565]]
[1.         0.12157565]
4 

[[-1.        -0.8708236]

[[-0.07418233 -1.        ]]
[-0.07418233 -1.        ]
13 

[[-0.12295546 -1.        ]]
[-0.12295546 -1.        ]
13 

[[-0.17740737 -0.9775055 ]]
[-0.17740737 -0.9775055 ]
13 

[[-0.26092598 -0.8065445 ]]
[-0.26092598 -0.8065445 ]
13 

[[-0.49751282 -0.04892502]]
[-0.49751282 -0.04892502]
13 

[[-0.72922045  1.        ]]
[-0.72922045  1.        ]
13 

[[-0.4813813  1.       ]]
[-0.4813813  1.       ]
13 

[[-0.5059076  1.       ]]
[-0.5059076  1.       ]
13 

[[-0.53326625  1.        ]]
[-0.53326625  1.        ]
13 

[[-0.56420165  1.        ]]
[-0.56420165  1.        ]
13 

[[-0.5995186  1.       ]]
[-0.5995186  1.       ]
13 

[[-0.6399778  1.       ]]
[-0.6399778  1.       ]
13 

[[-0.68609375  1.        ]]
[-0.68609375  1.        ]
13 

[[-0.7378203  1.       ]]
[-0.7378203  1.       ]
13 

[[-0.7942234  1.       ]]
[-0.7942234  1.       ]
13 

[[-0.8534339  1.       ]]
[-0.8534339  1.       ]
13 

[[-0.9132067  1.       ]]
[-0.9132067  1.       ]
13 

[[-0.9719637  1.       ]]
[-0

[[-0.560333  1.      ]]
[-0.560333  1.      ]
13 

[[-0.5794299  1.       ]]
[-0.5794299  1.       ]
13 

[[-0.5997779  1.       ]]
[-0.5997779  1.       ]
13 

[[-0.6213005  1.       ]]
[-0.6213005  1.       ]
13 

[[-0.6438484  1.       ]]
[-0.6438484  1.       ]
13 

[[-0.66722405  1.        ]]
[-0.66722405  1.        ]
13 

[[-0.6912253  1.       ]]
[-0.6912253  1.       ]
13 

[[-0.715706  1.      ]]
[-0.715706  1.      ]
13 

[[-0.7406479  1.       ]]
[-0.7406479  1.       ]
13 

[[-0.7662408  1.       ]]
[-0.7662408  1.       ]
13 

[[-0.7929577  1.       ]]
[-0.7929577  1.       ]
13 

[[-0.8216197  1.       ]]
[-0.8216197  1.       ]
13 

[[-0.8534387  1.       ]]
[-0.8534387  1.       ]
13 

[[-0.89003545  1.        ]]
[-0.89003545  1.        ]
13 

[[-0.93342024  1.        ]]
[-0.93342024  1.        ]
13 

[[-0.9859008  1.       ]]
[-0.9859008  1.       ]
13 

[[-1.  1.]]
[-1.  1.]
13 

[[-1.  1.]]
[-1.  1.]
13 

[[-1.         0.9274052]]
[-1.         0.9274052]
13 

[[-1.  

[[-0.3516734 -1.       ]]
[-0.3516734 -1.       ]
13 

[[-0.26990384 -1.        ]]
[-0.26990384 -1.        ]
13 

[[-0.18719628 -1.        ]]
[-0.18719628 -1.        ]
13 

[[-0.10992174 -1.        ]]
[-0.10992174 -1.        ]
13 

[[-0.04454756 -1.        ]]
[-0.04454756 -1.        ]
13 

[[ 0.00533593 -1.        ]]
[ 0.00533593 -1.        ]
13 

[[ 0.02689925 -1.        ]]
[ 0.02689925 -1.        ]
13 

[[-0.00290066 -1.        ]]
[-0.00290066 -1.        ]
13 

[[ 0.06936824 -0.967227  ]]
[ 0.06936824 -0.967227  ]
13 

[[-0.11714846 -0.85342485]]
[-0.11714846 -0.85342485]
13 

[[-0.10508232 -0.54039615]]
[-0.10508232 -0.54039615]
13 

[[-0.52538496 -0.60345244]]
[-0.52538496 -0.60345244]
13 

[[-0.46135992 -0.69549173]]
[-0.46135992 -0.69549173]
13 

[[-0.37021422 -0.83757836]]
[-0.37021422 -0.83757836]
13 

[[-0.24186146 -1.        ]]
[-0.24186146 -1.        ]
13 

[[-0.11605516 -1.        ]]
[-0.11605516 -1.        ]
13 

[[-0.13317655 -1.        ]]
[-0.13317655 -1.        ]
13 

[

[[-0.50571233  1.        ]]
[-0.50571233  1.        ]
13 

[[-0.5208861  1.       ]]
[-0.5208861  1.       ]
13 

[[-0.5370105  1.       ]]
[-0.5370105  1.       ]
13 

[[-0.5542566  1.       ]]
[-0.5542566  1.       ]
13 

[[-0.5727429  1.       ]]
[-0.5727429  1.       ]
13 

[[-0.5925174  1.       ]]
[-0.5925174  1.       ]
13 

[[-0.61354613  1.        ]]
[-0.61354613  1.        ]
13 

[[-0.63571155  1.        ]]
[-0.63571155  1.        ]
13 

[[-0.6588269  1.       ]]
[-0.6588269  1.       ]
13 

[[-0.6826698  1.       ]]
[-0.6826698  1.       ]
13 

[[-0.70703334  1.        ]]
[-0.70703334  1.        ]
13 

[[-0.73179  1.     ]]
[-0.73179  1.     ]
13 

[[-0.7569637  1.       ]]
[-0.7569637  1.       ]
13 

[[-0.78280103  1.        ]]
[-0.78280103  1.        ]
13 

[[-0.8098311  1.       ]]
[-0.8098311  1.       ]
13 

[[-0.83890605  1.        ]]
[-0.83890605  1.        ]
13 

[[-0.8712184  1.       ]]
[-0.8712184  1.       ]
13 

[[-0.908292  1.      ]]
[-0.908292  1.      ]
13 

[[-1.         0.4414056]]
[-1.         0.4414056]
13 

[[-1.          0.09235571]]
[-1.          0.09235571]
13 

[[-1.         -0.43410146]]
[-1.         -0.43410146]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-0.98944384 -1.        ]]
[-0.98944384 -1.        ]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-0.97958535 -1.        ]]
[-0.97958535 -1.        ]
13 

[[-0.9325058 -1.       ]]
[-0.9325058 -1.       ]
13 

[[-0.868941 -1.      ]]
[-0.868941 -1.      ]
13 

[[-0.792912 -1.      ]]
[-0.792912 -1.      ]
13 

[[-0.71131593 -1.        ]]
[-0.71131593 -1.        ]
13 

[[-0.6301728 -1.       ]]
[-0.6301728 -1.       ]
13 

[[-0.55219835 -1.        ]]
[-0.55219835 -1.        ]
13 

[[-0.4762804 -1.       ]]
[-0.4762804 -1.       ]
13 

[[-0.39895868 -1.        ]]
[-0.39895868 -1.        ]
13 

[[-0.31811637 -1.        ]]
[-0.31811637 -1.        ]
13 

[[-0.23613802 -1.

[[-0.7633529  1.       ]]
[-0.7633529  1.       ]
13 

[[-0.78930825  1.        ]]
[-0.78930825  1.        ]
13 

[[-0.81652707  1.        ]]
[-0.81652707  1.        ]
13 

[[-0.8458908  1.       ]]
[-0.8458908  1.       ]
13 

[[-0.8786053  1.       ]]
[-0.8786053  1.       ]
13 

[[-0.9161852  1.       ]]
[-0.9161852  1.       ]
13 

[[-0.960409  1.      ]]
[-0.960409  1.      ]
13 

[[-1.  1.]]
[-1.  1.]
13 

[[-1.  1.]]
[-1.  1.]
13 

[[-1.         0.9771541]]
[-1.         0.9771541]
13 

[[-1.          0.83029675]]
[-1.          0.83029675]
13 

[[-1.         0.4967687]]
[-1.         0.4967687]
13 

[[-1.          0.13926397]]
[-1.          0.13926397]
13 

[[-1.        -0.3329454]]
[-1.        -0.3329454]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-0.9955085 -1.       ]]
[-0.9955085 -1.       ]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-0.98931307 -1.        ]]
[-

[[-0.82624465  1.        ]]
[-0.82624465  1.        ]
13 

[[-0.85650414  1.        ]]
[-0.85650414  1.        ]
13 

[[-0.8905427  1.       ]]
[-0.8905427  1.       ]
13 

[[-0.9299514  1.       ]]
[-0.9299514  1.       ]
13 

[[-0.9765578  1.       ]]
[-0.9765578  1.       ]
13 

[[-1.  1.]]
[-1.  1.]
13 

[[-1.  1.]]
[-1.  1.]
13 

[[-1.          0.94370145]]
[-1.          0.94370145]
13 

[[-1.          0.73596525]]
[-1.          0.73596525]
13 

[[-1.          0.36246073]]
[-1.          0.36246073]
13 

[[-1.          0.01825627]]
[-1.          0.01825627]
13 

[[-1.         -0.57069016]]
[-1.         -0.57069016]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-1. -1.]]
[-1. -1.]
13 

[[-0.98074824 -1.        ]]
[-0.98074824 -1.        ]
13 

[[-0.929444 -1.      ]]
[-0.929444 -1.      ]
13 

[[-0.8623577 -1.       ]]
[-0.8623577 -1. 

In [55]:
plt.cla()
plt.plot(x_pose, y_pose, '+g')
plt.plot(cx, cy, "+b")
#plt.xlim([0, 25])
#plt.ylim([-20, 7])

In [ ]:
obs = husky_ppo.get_observation()

In [ ]:
print(obs)

In [ ]:
print(husky_ppo.get_pose())

In [ ]:
husky_ppo.closest_idx

In [ ]:
2*math.pi - 6.1321

In [ ]:
i = 0
while i < len(obs)-3:
    print([obs[i], obs[i+1], obs[i+2], obs[i+3]])
    i = i + 4

In [ ]:
obs2 = [ 3.1718987e-01,  2.6397274e+00, -7.1165800e-02,  2.9253261e+00,
         8.2946199e-01,  2.4810174e+00, -2.3757100e-01,  2.9868939e+00,
         1.4194126e+00,  2.3704438e+00, -3.9009976e-01,  2.9857869e+00,
         2.0244434e+00,  2.2775273e+00, -4.8208833e-01,  3.2309980e+00,
         2.5908678e+00,  2.2095571e+00, -5.5408025e-01,  3.0272779e+00,
         3.1385303e+00,  2.1544523e+00, -5.6134295e-01,  2.8097680e+00,
         3.6512144e+00,  2.1172132e+00, -5.9754920e-01,  2.6893539e+00,
         4.2099533e+00,  2.0821514e+00, -6.1818290e-01,  2.6712611e+00,
         4.7472596e+00,  2.0539546e+00, -6.3812470e-01,  2.9293561e+00,
         5.2431216e+00,  2.0312028e+00, -6.4151406e-01,  2.9405370e+00,
         3.7776546e-03, -1.4553810e-03]

In [ ]:
i = 0
while i < len(obs2)-3:
    print([obs2[i], obs2[i+1], obs2[i+2], obs2[i+3]])
    i = i + 4

In [ ]:
print(husky_ppo.num_waypoints)

In [20]:
print(husky_ppo.waypoints_list)

[array([0.        , 0.        , 5.84690706, 4.        ]), array([ 0.43545074, -0.20302535,  5.84595374,  4.        ]), array([ 0.87129576, -0.40674059,  5.84405479,  4.        ]), array([ 1.30792936, -0.61183558,  5.8412255 ,  4.        ]), array([ 1.74574581, -0.81900022,  5.83748844,  4.        ]), array([ 2.1851394 , -1.02892438,  5.83287314,  4.        ]), array([ 2.62650443, -1.24229795,  5.82741558,  4.        ]), array([ 3.07023516, -1.4598108 ,  5.82115759,  4.        ]), array([ 3.5167259 , -1.68215281,  5.81414619,  4.        ]), array([ 3.96637093, -1.91001388,  5.80643284,  4.        ]), array([ 4.41956452, -2.14408387,  5.79807264,  4.        ]), array([ 4.87670098, -2.38505267,  5.78912349,  4.        ]), array([ 5.33817457, -2.63361015,  5.77964528,  4.        ]), array([ 5.8043796 , -2.89044621,  5.77000864,  4.        ]), array([ 6.27566617, -3.15603351,  5.76509605,  4.        ]), array([ 6.75167852, -3.42737363,  5.76856828,  4.        ]), array([ 7.23149358, -3.6986